### **ETL - steam_games**
**En este notebook realizaremos la extracción, transformación y carga (ETL) de `steam_games`**

In [34]:
# Importando librerías
import json                     # Módulo de codificador y decodificador JSON
import pandas as pd             # Librería para manipular datasets
import pyarrow as pa            # Útil para realizar operaciones de lectura y escritura de datos
import pyarrow.parquet as pq    # Útil para leer y escribir datos en formato Parquet de manera eficiente 
import os                       # creación de directorios y comprobación de existencia
import re                       # 

#### **Extracción y exploración**

In [35]:
#Se crea una lista sin elementos con el propósito de almacenar el registro de iteraciones que ocurrirán en el bucle for.
contenido = []

#Creamos bucle que va a recorrer el dataset.
with open ("dataset/output_steam_games.json") as file:
    for line in file.readlines():
        fila = json.loads(line)
        contenido.append(fila)

#Se crea el dataframe en base a la lista
steam = pd.DataFrame(contenido)
steam

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [36]:
# Obtenemos la forma de la matriz
steam.shape

(120445, 13)

In [37]:
steam.describe()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
count,24083,28852,32133,30085,32135,30068,31972,32133,31465,30758.00,32135,32133,28836
unique,8239,883,32094,30054,32135,3582,15395,32132,4649,162.00,2,32132,10992
top,Ubisoft,[Action],Soundtrack,Soundtrack,http://store.steampowered.com/app/761140/Lost_...,2012-10-16,"[Casual, Simulation]",http://steamcommunity.com/app/612880/reviews/?...,[Single-player],4.99,False,612880,Ubisoft - San Francisco
freq,385,1880,3,3,1,100,1292,2,2794,4278.00,30188,2,1259


In [38]:
steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


Busqueda y tratamiento de nulos y duplicados

In [39]:
# Busqueda de nulos
Total_nulos= steam.isnull().sum()
Total_nulos

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Existen columnas que no se utilizan por lo tanto se eliminan con el fin de optimizar el tratamiento de los datos

In [40]:
steam.drop(['title','early_access','url','reviews_url','tags','publisher','specs'], axis=1, inplace = True)
steam.reindex()

,genres,app_name,release_date,price,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns"


In [41]:
# Borrar nulos por cada columna
steam = steam.dropna(subset=['genres'])
steam = steam.dropna(subset=['app_name'])
steam = steam.dropna(subset=['release_date'])
steam = steam.dropna(subset=['price'])
steam = steam.dropna(subset=['id'])
steam = steam.dropna(subset=['developer'])

In [42]:
# Reseteamos index
steam= steam.reset_index(drop=True)

In [43]:
# visualizamos duplicados de "Machine Games"
steam[steam["developer"]== "Machine Games"]

,genres,app_name,release_date,price,id,developer
10497,[Action],Wolfenstein II: The Freedom Chronicles - Seaso...,2017-11-07,24.99,650410,Machine Games
11359,[Action],Wolfenstein II: The Freedom Chronicles - Episo...,2017-12-13,9.99,624620,Machine Games
11810,[Action],Wolfenstein II: The New Colossus,2017-10-26,59.99,612880,Machine Games
12361,[Action],Wolfenstein II: The New Colossus,2017-10-26,59.99,612880,Machine Games
24543,[Action],Wolfenstein: The New Order,2014-05-19,19.99,201810,Machine Games


La fila duplicada 12361 es idéntica a la 11810.. por lo que podemos eliminarla

In [44]:
#elimino columnas con id duplicadas
duplicados_col = [12361]
steam= steam.drop(duplicados_col)

In [45]:
steam.shape

(27461, 6)

#### **Tranformación de los Datos**

Observamos que tipo de datos es cada una de las columnas

In [46]:
data_type_process = {
    "columna": list(steam.columns),
    "tipos_de_datos": [steam[columna].apply(type).unique() for columna in steam.columns]
}

data_type= pd.DataFrame(data_type_process)
data_type

,columna,tipos_de_datos
0,genres,[<class 'list'>]
1,app_name,[<class 'str'>]
2,release_date,[<class 'str'>]
3,price,"[<class 'float'>, <class 'str'>]"
4,id,[<class 'str'>]
5,developer,[<class 'str'>]


**Tranformación de los datos de la columna `app name`**

In [47]:
# Visualizamos la columna "app_name"
steam["app_name"]

0             Lost Summoner Kitty
1                       Ironbound
2         Real Pool 3D - Poolians
3                         弹炸人2222
4           Battle Royale Trainer
                   ...           
27457                Kebab it Up!
27458              Colony On Mars
27459    LOGistICAL: South Africa
27460               Russian Roads
27461         EXIT 2 - Directions
Name: app_name, Length: 27461, dtype: object

Se normaliza los nombres de desarrolladores en la columna `app_name` del DataFrame `games`, eliminando caracteres no alfabéticos, convirtiendo letras a minúsculas y eliminando espacios adicionales.

In [48]:
def normalize_developer_name(name):
    if isinstance(name, str):  
        cleaned_name = re.sub(r'[^a-zA-Z\s]', '', name)
        normalized_name = re.sub(r'\s+', ' ', cleaned_name.lower().strip())
        return normalized_name
    else:
        return name

steam['app_name'] = steam['app_name'].apply(normalize_developer_name)

In [49]:
steam['app_name'] = steam['app_name'].str.title()

**Transformación de la columna `release_date`**

In [50]:
# Visualizamos la columna "release_date"
steam["release_date"]

0        2018-01-04
1        2018-01-04
2        2017-07-24
3        2017-12-07
4        2018-01-04
            ...    
27457    2018-01-04
27458    2018-01-04
27459    2018-01-04
27460    2018-01-04
27461    2017-09-02
Name: release_date, Length: 27461, dtype: object

In [51]:
# Calcular la frecuencia de valores únicos
steam["release_date"].value_counts()

release_date
2012-10-16    100
2017-08-31     90
2017-09-26     88
2017-07-25     78
2017-12-19     74
             ... 
2016-01-03      1
1998-01-29      1
2000-01-31      1
1994-08-15      1
2004-03-01      1
Name: count, Length: 3243, dtype: int64

Para extraer el año de la fecha registrada en la columna `release_date`, es necesaria una transformación. Esto implica crear una nueva columna que contenga únicamente el año de lanzamiento `release_year`

In [52]:
steam['release_year'] = steam['release_date'].str.extract(r'(\d{4})')
steam.head()

,genres,app_name,release_date,price,id,developer,release_year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool D Poolians,2017-07-24,Free to Play,670290,Poolians.com,2017
3,"[Action, Adventure, Casual]",,2017-12-07,0.99,767400,彼岸领域,2017
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,2018-01-04,3.99,772540,Trickjump Games Ltd,2018


In [53]:
#Convertimos a tipo de datos int
steam['release_year'] = steam['release_year'].fillna(0).astype('int64')

In [54]:
print(steam['release_year'].unique())

[2018 2017 1997 1998 2016 2006 2005 2003 2007 2002 2000 1995 1996 1994
 2001 1993 2004 2008 2009 1999 1992 1989 2010 2011 2013 2012 2014 1983
 1984 2015 1990 1988 1991 1987 1986    0 2021 2019 1985]


In [55]:
#excluyo el 0 de los datos
steam= steam[steam['release_year'] != 0]

**Transformación de los datos de la columna `price`**

In [56]:
#observo la composición de la columna
steam["price"]

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                3.99
             ...     
27457            1.99
27458            1.99
27459            4.99
27460            1.99
27461            4.99
Name: price, Length: 27458, dtype: object

In [26]:
# Calcular la frecuencia de valores únicos
steam["price"].value_counts()

price
4.99           3828
9.99           3568
2.99           3155
0.99           2458
1.99           2245
               ... 
179.0             1
6.66              1
189.0             1
1.5               1
Free to Use       1
Name: count, Length: 149, dtype: int64

In [57]:
steam['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 3.99, 9.99, 18.99,
       29.99, 10.99, 2.99, 1.59, 14.99, 1.99, 59.99, 8.99, 6.99, 7.99,
       39.99, 'Free', 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25, 24.99,
       17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo', 'Play for Free!',
       34.99, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99, 49.99, 5.0,
       13.98, 29.96, 109.99, 149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 'Free Mod',
       54.99, 64.99, 74.99, 'Install Theme', 0.89, 'Third-party', 0.5,
       'Play Now', 299.99, 1.29, 119.99, 44.99, 3.0, 15.0, 5.49, 23.99,
       49.0, 10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0,
       4.0, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 129.99, 179.0, 26.99,
       399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 22.99, 320.0, 38.85, 71.7,
       995.0, 27.49, 3.39, 6.0, 19.95, 20.99, 499.99, 199.99, 16.06, 4.68,
       131.4, 44.98, 202.76, 2.3, 0.95, 172.24, 249.99, 

Hay valores en la columna 'price' que contienen 'str', en su gran mayoria indican que son 'Free', los reemplazo por 0.0, luego cambio el tipo de dato a float

In [58]:
# Cambiar valores de la columna 'price' donde indique que sea "str" por '0.0'
steam['price'] = steam['price'].apply(lambda x: 0.0 if isinstance(x,str) else x)
steam['price'] = steam['price'].astype('float')

**Tratamiento de los datos de la columna `genres`**

In [59]:
steam["genres"][0]

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']

Al utilizar la función "explode", puedo extraer cada valor almacenado en la columna `genres` y mostrarlos en un nuevo campo conservando su id asociada original.

In [60]:
steam = steam.explode("genres")
steam = steam.dropna(subset=["genres"]).reset_index(drop=True)

In [61]:
# Convertir los valores de la columna 'genres' en una cadena separada por comas
steam['genres'] = steam['genres'].astype(str).apply(lambda x: ', '.join(filter(None, x.split(','))))

**Tranformación de datos de la columna `id`**

In [62]:
#observo la composición de la columna
steam["id"]

0        761140
1        761140
2        761140
3        761140
4        761140
          ...  
68011    610660
68012    610660
68013    610660
68014    658870
68015    658870
Name: id, Length: 68016, dtype: object

In [63]:
# cambia el nombre de la columna 'id' a 'item_id'
steam = steam.rename(columns={'id':'item_id'})

In [64]:
# se reordenan las variables con caracteristicas mas importantes a criterio
steam = steam[['item_id','app_name','genres','developer','price','release_year']]

In [65]:
#Reseteo el indice de las filas
steam.reset_index(drop=True, inplace=True)

**Almaceno el DataFrame en un formato de archivo Parquet con el fin de mejorar su eficiencia y facilitar su uso en consultas posteriores**

Se crea una copia independiente del DataFrame `steam` y asigna esta copia a la variable `steam_copy`. Esto se hace para evitar que cualquier modificación realizada en la copia afecte al DataFrame original, permitiendo trabajar con los datos de manera segura sin alterar el DataFrame original.

In [66]:
# Generamos copia
steam_copy = steam.copy()

#### **Guardo incialmente el DF en formato CSV, llamado `steam_games.csv`**

In [67]:
ruta_carpeta = "data"

if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

ruta_guardar_csv = 'data/steam_games.csv'
steam_copy.to_csv(ruta_guardar_csv, index=False, encoding='utf-8')

#### **Convierto el CSV y guardo como `steam_games.parquet`**

In [68]:
#Se lee el archivo csv
steam_copy= pd.read_csv("data/steam_games.csv") 

#Asigno la ruta donde quiero guardar el parquet con el nombre que va tener
ruta_guardar_parquet= "data/steam_games.parquet"

#Transformo a una tabla el archivo csv en parquet
table = pa.Table.from_pandas(steam_copy)
pq.write_table(table,ruta_guardar_parquet)

In [69]:
steam

,item_id,app_name,genres,developer,price,release_year
0,761140,Lost Summoner Kitty,Action,Kotoshiro,4.99,2018
1,761140,Lost Summoner Kitty,Casual,Kotoshiro,4.99,2018
2,761140,Lost Summoner Kitty,Indie,Kotoshiro,4.99,2018
3,761140,Lost Summoner Kitty,Simulation,Kotoshiro,4.99,2018
4,761140,Lost Summoner Kitty,Strategy,Kotoshiro,4.99,2018
...,...,...,...,...,...,...
68011,610660,Russian Roads,Indie,Laush Dmitriy Sergeevich,1.99,2018
68012,610660,Russian Roads,Racing,Laush Dmitriy Sergeevich,1.99,2018
68013,610660,Russian Roads,Simulation,Laush Dmitriy Sergeevich,1.99,2018
68014,658870,Exit Directions,Casual,"xropi,stev3ns",4.99,2017
